In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import pandas as pd, numpy as np
from crypto_trading.common.db import DatabaseHandler
from crypto_trading.analysis.features.feature_generator import *
from crypto_trading.analysis.features.normalize import *
from duckdb import sql
from pathlib import Path

In [3]:
pd.set_option('display.max_columns', 100)

# Config / Params

In [4]:
path_data = Path() / '..' / 'data'
path_data_raw = path_data / 'raw'
path_data_save = path_data / 'processed'

# Get Data

## Connect to Database

In [5]:
db = DatabaseHandler()

## Get OHLC Data

In [6]:
# Volume=Weighted Data by Instrument
query_ohlc_1d = """
    SELECT 
        instrument,
        datetime,
        -- high - low AS range,
        -- range / open AS range_perc,
        SUM(open * volume) / SUM(volume) AS open,
        SUM(high * volume) / SUM(volume) AS high, 
        SUM(low * volume) / SUM(volume) AS low,
        SUM(close * volume) / SUM(volume) AS close,
        SUM(volume) as volume,
        -- SUM(range * volume) / SUM(range) AS range,
        -- SUM(range_perc * volume) / SUM(range_perc) AS range_perc,

    FROM amberdata.ohlcv_perps_1d
    WHERE 
        CAST(datetime AS TIMESTAMP) >= NOW() - INTERVAL 102 DAY
        AND instrument IN (
            SELECT instrument
            FROM amberdata.exchange_reference
            WHERE
                exchange_enabled = TRUE
                AND contract_period = 'perpetual'
                AND quote_symbol = 'USDT'
            )
    GROUP BY instrument, datetime
    ORDER BY instrument ASC, datetime ASC
"""
df_ohlc_1d = (
    db.query_to_df(query_ohlc_1d)
    .assign(volume_usd = lambda x: x['volume'] * x[['open','high','low','close']].mean(axis=1))
)

In [7]:
df_ohlc_1d.sort_values('datetime', ascending=False)

,instrument,datetime,open,high,low,close,volume,volume_usd
50084,ZRXUSDT,2025-02-17,0.326243,0.347478,0.321722,0.331878,7.725358e+07,2.563509e+07
18861,FOXYUSDT,2025-02-17,0.005525,0.005746,0.005214,0.005280,9.890828e+08,5.381847e+06
35783,PUFFERUSDT,2025-02-17,0.417400,0.422400,0.383600,0.390000,4.876187e+06,1.966810e+06
18759,FORTHUSDT,2025-02-17,3.654200,3.703500,3.413400,3.478000,2.475429e+05,8.818159e+05
9286,BILLYUSDT,2025-02-17,0.003600,0.003775,0.003162,0.003196,4.858665e+07,1.668186e+05
...,...,...,...,...,...,...,...,...
15329,DOGEUSDT,2024-11-08,0.193340,0.207130,0.190510,0.202030,1.475681e+10,2.925574e+09
45055,UNIUSDT,2024-11-08,8.904000,9.054000,8.524000,8.861000,1.741924e+07,1.539121e+08
15227,DODOXUSDT,2024-11-08,0.114137,0.114998,0.108535,0.112524,3.843310e+07,4.325588e+06
15125,DODOUSDT,2024-11-08,0.114090,0.115080,0.108670,0.112550,3.667210e+06,4.129187e+05


In [8]:
query_highest_volume = """
WITH LatestDatePerInstrument AS (
    SELECT
        ohlcv.instrument,
        MAX(ohlcv.datetime) AS latest_datetime
    FROM
        amberdata.ohlcv_perps_1d ohlcv
    JOIN
        amberdata.ohlcv_info_futures info
    ON
        ohlcv.exchange = info.exchange
        AND ohlcv.instrument = info.instrument
    JOIN
        amberdata.exchange_reference ref
    ON
        ohlcv.exchange = ref.exchange
        AND ohlcv.instrument = ref.instrument
    WHERE
        info.active = true
        AND ref.exchange_enabled = true
        AND ref.quote_symbol = 'USDT'
    GROUP BY
        ohlcv.instrument
),
VolumeRanked AS (
    SELECT
        ohlcv.instrument,
        ohlcv.exchange,
        ohlcv.volume,
        ROW_NUMBER() OVER (PARTITION BY ohlcv.instrument ORDER BY ohlcv.volume DESC) as volume_rank
    FROM
        amberdata.ohlcv_perps_1d ohlcv
    JOIN
        LatestDatePerInstrument ldi
    ON
        ohlcv.instrument = ldi.instrument
        AND ohlcv.datetime = ldi.latest_datetime
    JOIN
        amberdata.ohlcv_info_futures info
    ON
        ohlcv.exchange = info.exchange
        AND ohlcv.instrument = info.instrument
    JOIN
        amberdata.exchange_reference ref
    ON
        ohlcv.exchange = ref.exchange
        AND ohlcv.instrument = ref.instrument
    WHERE
        info.active = true
        AND ref.exchange_enabled = true
)
SELECT
    instrument,
    exchange,
    volume
FROM
    VolumeRanked
WHERE
    volume_rank = 1
ORDER BY
    volume DESC;
"""
df_highest_volume = db.query_to_df(query_highest_volume)

In [9]:
df_highest_volume.sort_values(['instrument', 'exchange']).iloc[50:60]

,instrument,exchange,volume
476,AKTUSDT,binance,1.118360e+06
53,ALCHUSDT,binance,1.284863e+09
452,ALEOUSDT,bybit,2.142126e+06
171,ALGOUSDT,binance,1.406054e+08
383,ALICEUSDT,binance,8.858617e+06
230,ALPACAUSDT,binance,6.309696e+07
202,ALPHAUSDT,binance,9.357497e+07
156,ALTUSDT,binance,1.780439e+08
341,ALUUSDT,bybit,1.673445e+07
8,AMBUSDT,binance,2.365143e+10


# Calculate Features

In [10]:
features = [
{
        "name": "range_perc",
        "func": calc_range_percentage,
        "params": {},  # Uses default column names
        "group_by_instrument": True,  # Needs to be grouped as it uses OHLC data
    },
    {
        "name": "range_perc_ema_5",
        "func": calc_ema,
        "params": {
            "col": "range_perc",
            "lookback": 5,
        },
        "group_by_instrument": True,  # EMAs need to be calculated per instrument
    },
    {
        "name": "range_perc_ema_5_log",
        "func": lambda df, col: np.log(df[col]),
        "params": {
            "col": "range_perc_ema_5"
        },
        "group_by_instrument": False,  # Simple transformation, can be applied to all data at once
    },
    {
        "name": "range_perc_log_pred",
        "func": lambda df, col: df[col] * 0.82113026 - 0.5240189478074768,
        "params": {
            "col": "range_perc_ema_5_log"
        },
        "group_by_instrument": False,  # Simple arithmetic, can be applied to all data at once
    },
    {
        "name": "range_perc_pred",
        "func": lambda df, col: np.exp(df[col]),
        "params": {
            "col": "range_perc_log_pred"
        },
        "group_by_instrument": False,  # Simple transformation, can be applied to all data at once
    },
    {
        "name": "range_perc_pred_prev",
        "func": shift_values,
        "params": {
            "col": "range_perc_pred",
            "shift": 1
        },
        "group_by_instrument": True,  # Shifting needs to be done per instrument
    },
    {
        "name": "range_perc_rel",
        "func": lambda df, col1, col2: np.log(df[col1] / df[col2]),
        "params": {
            "col1": "range_perc",
            "col2": "range_perc_pred_prev"
        },
        "group_by_instrument": False,  # Simple arithmetic, can be applied to all data at once
    },
    {
        'name':'three_bar_triangle',
        'func':get_bar_triangle,
        'params':{'lookback':3},
    },
    {
        'name':'high_3d',
        'func':get_highest_high,
        'params':{'lookback':3},
    },
    {
        'name':'low_3d',
        'func':get_lowest_low,
        'params':{'lookback':3},
    },
]

# Position in Range Features
for lookback in [3,7,10,30,100]:
    params_pos_in_range = {
        'name':f'pos_in_range_{lookback}',
        'func':calc_pos_in_range,
        'params':{'lookback':lookback},
    }
    features.append(params_pos_in_range)

# EMA Cross Features
for lookback_pair in [(3,12), (10,40), (25,100)]:
    fast_lookback = lookback_pair[0]
    slow_lookback = lookback_pair[1]
    params_cmema = {
        'name':f'cmema_{fast_lookback}_{slow_lookback}',
        'func':calc_cmema,
        'params':{
            'fast_lookback':fast_lookback,
            'slow_lookback':slow_lookback,
            "normalize_by": "range_perc_ema_5",
        },
    }
    features.append(params_cmema)

# Aroon Features
for lookback in [10, 30, 60]:
    params_aroon = {
        'name':f'aroon_{lookback}',
        'func':calc_aroon,
        'params':{'lookback':lookback},
    }
    features.append(params_aroon)

In [11]:
import time

start_time = time.time()
df_features = generate_features(df_ohlc_1d, features, 'instrument')
print(f'Ran in {time.time() - start_time:.2f}s')

/home/quantfiction/repositories/crypto_trading/crypto_trading/analysis/features/feature_generator.py:168: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(windows, axis=1), np.nanmin(windows, axis=1)
/home/quantfiction/repositories/crypto_trading/crypto_trading/analysis/features/feature_generator.py:168: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(windows, axis=1), np.nanmin(windows, axis=1)
/home/quantfiction/repositories/crypto_trading/crypto_trading/analysis/features/feature_generator.py:168: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(windows, axis=1), np.nanmin(windows, axis=1)
/home/quantfiction/repositories/crypto_trading/crypto_trading/analysis/features/feature_generator.py:168: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(windows, axis=1), np.nanmin(windows, axis=1)
/home/quantfiction/repositories/crypto_trading/crypto_trading/analysis/features/feature_generator.py:168: RuntimeWarning: All-NaN slice encountered
  re

Ran in 2.71s


In [46]:
last_date = df_features['datetime'].max()
df_recent = df_features.query('datetime == @last_date and volume > 0')
df_recent = (
    pd.merge(
        df_recent,
        df_highest_volume.drop('volume', axis='columns'),
        on='instrument',
        how='left'
    )
    .fillna({'instrument':'binance'})
)

# Big Boys

In [66]:
short_list = [
    'BTCUSDT',
    'ETHUSDT',
    'SOLUSDT',
    'BNBUSDT',
    'DOGEUSDT',
    'TONUSDT',
    'XRPUSDT',
]

# Filter on instruments and normalize columns with different bounds
df_filtered = df_recent[df_recent['instrument'].isin(short_list)]

# Normalize aroon columns (scale: -100 to 100) and cmema (scale: -2 to 2)
df_normalized = (
    df_filtered
    .pipe(normalize_0_1, cols=['aroon_10', 'aroon_30'], min_value=-100, max_value=100)
    .pipe(normalize_0_1, cols=['cmema_3_12'], min_value=-2, max_value=2)
)

# Compute an aggregate mean for trend features
df_normalized = df_normalized.assign(
    mean_st_trend=lambda x: x[[
        'pos_in_range_3',
        'pos_in_range_7',
        'pos_in_range_30',
        'aroon_10',
        'aroon_30',
        'cmema_3_12',
    ]].mean(axis=1)
)

# Select, sort, and index the desired columns
cols_order = [
    'instrument',
    'range_perc_rel',
    'mean_st_trend',
    'pos_in_range_3',
    'pos_in_range_7',
    'pos_in_range_30',
    'aroon_10',
    'aroon_30',
    'cmema_3_12'
]
df_big_boys = (
    df_normalized[cols_order]
    .sort_values('instrument')
    .set_index('instrument')
)

# Append an ‘Average’ row (alternatively, use pd.concat for immutability)
df_big_boys.loc['Average'] = df_big_boys.mean()

In [15]:
(
    df_recent
    .loc[
         df_recent['instrument'].isin(short_list),
        [
            'instrument',
            'range_perc_pred',
        ]
    ]
).to_csv(path_data_save / 'big_boys.csv')

# Relative Ranges

In [16]:
range_perc_rel_recent = (
    df_recent['range_perc_rel']
)

# Current Feature Values

In [17]:
current_feature_values = (
    df_recent
    [[
        'datetime',
        'pos_in_range_3',
        'pos_in_range_10',
        'pos_in_range_30',
        'cmema_3_12',
        'cmema_10_40',
        'cmema_25_100',
        'aroon_10',
        'aroon_30',
        'aroon_60',
    ]]
    # .query('datetime == "2024-04-08"')
    # .sort_values(['datetime', 'pos_in_range_30d'], ascending=False)
    # # .head(30)
    .drop(['datetime'], axis='columns')
)

# Set Biases

In [18]:
current_feature_values.isnull().any()

pos_in_range_3     False
pos_in_range_10     True
pos_in_range_30     True
cmema_3_12          True
cmema_10_40         True
cmema_25_100        True
aroon_10           False
aroon_30           False
aroon_60           False
dtype: bool

In [19]:
current_feature_values

,pos_in_range_3,pos_in_range_10,pos_in_range_30,cmema_3_12,cmema_10_40,cmema_25_100,aroon_10,aroon_30,aroon_60
101,0.573529,0.647668,0.155673,-0.576096,-5.115328,-7.138335,55.555556,-51.724138,-50.847458
203,0.380013,0.358841,0.289586,-0.072344,-1.975765,-2.582819,44.444444,-51.724138,-74.576271
288,0.083700,0.218714,0.269765,0.048648,0.536892,0.463218,-11.111111,72.413793,44.067797
390,0.368428,0.408239,0.185312,-0.359676,-2.316160,-2.779239,33.333333,-51.724138,-47.457627
492,0.911807,0.954711,0.326428,0.303602,-2.235764,-3.225194,100.000000,-51.724138,-74.576271
...,...,...,...,...,...,...,...,...,...
49693,0.349169,0.624658,0.861125,0.657585,2.630458,8.417256,66.666667,65.517241,96.610169
49795,0.385112,0.617246,0.300509,-0.335715,-2.564730,-2.741799,55.555556,-51.724138,-47.457627
49880,0.574225,0.910952,0.787144,1.289569,-0.385630,-1.143320,66.666667,-51.724138,-35.593220
49982,0.303025,0.572320,0.312961,-0.102998,-1.991128,-2.424316,66.666667,-51.724138,-74.576271


In [20]:
# Big Boys Rel. Range
bb_rel_range = 1 - min_max_norm(df_big_boys['range_perc_rel'].drop('Average').agg(['median','mean']), -1.5, 1.5)

# Big Boys Short-Term Trend
bb_st_trend = df_big_boys['mean_st_trend'].drop('Average').agg(['median','mean'])

# All Rel. Range
all_rel_range = 1 - min_max_norm(range_perc_rel_recent.agg(['mean','median']).mean(), min_value=-1.5, max_value=1.5)

# All Short-Term Trend
all_st_trend = (
    current_feature_values
    .pipe(normalize_0_1, cols=['aroon_10', 'aroon_30'], min_value=-100, max_value=100)
    .pipe(normalize_0_1, cols=['cmema_3_12'], min_value=-2, max_value=2)
    [[
        'pos_in_range_3',
        'pos_in_range_10',
        'pos_in_range_30',
        'aroon_10',
        'aroon_30',
        'cmema_3_12',
    ]]
    .mean(axis=1)
    .agg(['median','mean'])
    # .mean(axis=1)
    # .mean()
)

avg_rel_range = np.mean([bb_rel_range.mean(), all_rel_range])
avg_st_trend = np.mean([bb_st_trend.mean(), all_st_trend.mean()])

In [21]:
biases = (
    pd.Series({
        'Bull':avg_st_trend,
        'Bear':1-avg_st_trend,
        'Trend':avg_rel_range,
        'Chop':1-avg_rel_range,
    })
    .to_frame()
)

print(last_date)
biases

2025-02-17 00:00:00


,0
Bull,0.395433
Bear,0.604567
Trend,0.481932
Chop,0.518068


# Watchlists

## Breakouts

In [22]:
df_highest_volume
dict_highest_volume = dict(zip(df_highest_volume['instrument'], df_highest_volume['exchange']))

In [48]:
filter_1 = (
    df_recent
    .query('three_bar_triangle == True and pos_in_range_3 >= 0.67 and range_perc_rel < 0')
)

filter_2 = (
    df_recent
    .query('aroon_10 >= 50 and aroon_30 >= 50')
)

breakouts = (
    pd.merge(filter_1, filter_2[['instrument']], on='instrument')
    .assign(high_distance = lambda x: (x['high_3d'] / x['close'] - 1) / x['range_perc_pred'])
    .sort_values(['pos_in_range_30','aroon_30'], ascending=False)
    [[
        'exchange',
        'instrument',
        'range_perc_pred',
        'range_perc_rel',
        'high_distance',
    ]]
    .reset_index(drop=True)
)

# Save CSV
breakouts.to_csv(path_data_save / 'breakouts.csv')

# Save Text Files
txt_breakouts = ','.join([f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in breakouts['instrument']])
with open(path_data_raw / 'Breakouts.txt', 'w') as f:
    f.write(txt_breakouts)

print('Breakouts')
print('='*20)
breakouts

Breakouts


,exchange,instrument,range_perc_pred,range_perc_rel,high_distance


## Breakdowns

In [49]:
filter_3 = (
    df_recent
    .assign(low_distance = lambda x: -(x['low_3d'] / x['close'] - 1) / x['range_perc_pred'])
    .query('three_bar_triangle == True and pos_in_range_3 <= 0.4 and range_perc_rel < 0 and low_distance < 0.5')
)

filter_4 = (
    df_recent
    # .assign(range_perc_ema_5_prev = lambda x: get_groupby_vals(x, 'instrument', shift_values, kwargs={'col':'range_perc_ema_5', 'shift':1}))
    # .assign(range_perc_rel = lambda x: np.log(x['range_perc'] / x['range_perc_ema_5_prev']))
    .query('aroon_10 <= -50 and aroon_30 <= 0')
)

breakdowns = (
    pd.merge(filter_3, filter_4[['instrument']], on='instrument')
    .sort_values(['range_perc_rel', 'low_distance', 'pos_in_range_30','aroon_30'], ascending=True)
    [[
        'exchange',
        'instrument',
        'range_perc_pred',
        'low_distance',
    ]]
    .reset_index(drop=True)
)

# Save CSV
breakdowns.to_csv(path_data_save / 'breakdowns.csv')

# Save Text File
txt_breakdowns = ','.join([f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in breakdowns['instrument']])
with open(path_data_raw / 'Breakdowns.txt', 'w') as f:
    f.write(txt_breakdowns)

print('Breakdowns')
print('='*20)
breakdowns

Breakdowns


,exchange,instrument,range_perc_pred,low_distance
0,binance,HIPPOUSDT,0.054103,0.224945


## Rip Fades

In [50]:
filter_5 = (
    df_recent
    .query('range_perc_rel >= 0.25')
)

rip_fades = (
    pd.merge(filter_5, filter_4[['instrument']], on='instrument')
    .sort_values(['range_perc_rel', 'pos_in_range_30','aroon_30'], ascending=[False, True, True])
    [[
        'exchange',
        'instrument',
        'range_perc_rel',
        'range_perc_pred',
        'pos_in_range_30',
        'aroon_30'
    ]]
    .reset_index(drop=True)
)

# Save CSV
rip_fades.to_csv(path_data_save / 'rip_fades.csv')

# Save Text File
txt_rip_fades = ','.join([f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in rip_fades['instrument']])
with open(path_data_raw / 'Rip Fades.txt', 'w') as f:
    f.write(txt_rip_fades)

print('Rip Fades')
print('='*20)
rip_fades

Rip Fades


,exchange,instrument,range_perc_rel,range_perc_pred,pos_in_range_30,aroon_30
0,binance,AMBUSDT,1.557643,0.220536,0.020156,-100.000000
1,bybit,FWOGUSDT,0.926198,0.153796,0.005666,-93.103448
2,bybit,NSUSDT,0.859257,0.188608,0.136795,-72.413793
3,bybit,SWEATUSDT,0.738798,0.107408,0.033620,-55.172414
4,bybit,PEAQUSDT,0.695759,0.104918,0.008967,-100.000000
5,binance,FARTCOINUSDT,0.653804,0.190972,0.004130,-100.000000
6,binance,DYMUSDT,0.632467,0.092355,0.017199,-100.000000
7,bybit,GEMSUSDT,0.601691,0.111625,0.103402,-31.034483
8,bybit,JUSDT,0.570931,0.110411,NaN,0.000000
9,bybit,1000MUMUUSDT,0.563903,0.190638,0.006445,-100.000000


## Dip Buys

In [51]:
dip_buys = (
    pd.merge(filter_5, filter_2[['instrument']], on='instrument')
    .sort_values(['range_perc_rel', 'pos_in_range_30','aroon_30'], ascending=[False, False, False])
    [[
        'exchange',
        'instrument',
        'range_perc_rel',
        'range_perc_pred',
        'pos_in_range_30',
        'aroon_30',
    ]]
    .reset_index(drop=True)
)

# Save CSV
dip_buys.to_csv(path_data_save / 'dip_buys.csv')

# Save Text File
txt_dip_buys = ','.join([f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in dip_buys['instrument']])
with open(path_data_raw / 'Dip Buys.txt', 'w') as f:
    f.write(txt_dip_buys)

print('Dip Buys')
print('='*20)
dip_buys

Dip Buys


,exchange,instrument,range_perc_rel,range_perc_pred,pos_in_range_30,aroon_30


## Short-Term Under/Out-Performers

In [52]:
st_trend_rank = (
    df_recent
    .set_index(['exchange', 'instrument'])
    [[
        'range_perc_pred',
        'pos_in_range_3',
        'pos_in_range_7',
        'pos_in_range_30',
        # 'pos_in_range_100d',
        'cmema_3_12',
        # 'cmema_10_40',
        # 'cmema_25_100',
        'aroon_10',
        'aroon_30',
        # 'aroon_100',
    ]]
    # .rank()
    .assign(average_score = lambda x: x.rank(ascending=False).mean(axis=1))
    .sort_values('average_score', ascending=True)
    .dropna()
    .reset_index()
)

top_st_trend = st_trend_rank.head(20)
bottom_st_trend = st_trend_rank.tail(20)

# Save CSVs
top_st_trend.to_csv(path_data_save / 'top_st_trend.csv')
bottom_st_trend.to_csv(path_data_save / 'bottom_st_trend.csv')

# Save Text Files
txt_st_up = ','.join([f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in top_st_trend['instrument']])
with open(path_data_raw / 'Short-Term Outperformers.txt', 'w') as f:
    f.write(txt_st_up)

txt_st_down = ','.join([f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in bottom_st_trend['instrument']])
with open(path_data_raw / 'Short-Term Underperformers.txt', 'w') as f:
    f.write(txt_st_down)

## Long-Term Out-Performers

In [53]:
top_lt_trend = (
    df_recent
    .dropna(subset=['pos_in_range_100'])
    .query('pos_in_range_100 >= 0.75')
    .sort_values('pos_in_range_100', ascending=False)
    [[
        'exchange',
        'instrument',
        'range_perc_pred',
        'pos_in_range_100',
    ]]
    .head(20)
)

# Save CSV
top_lt_trend.to_csv(path_data_save / 'top_lt_trend.csv')

# Save Text File
txt_lt_up = ','.join([f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in top_lt_trend['instrument']])
with open(path_data_raw / 'Long-Term Outperformers.txt', 'w') as f:
    f.write(txt_lt_up)

## Long-Term Out-Performers

In [54]:
bottom_lt_trend = (
    df_recent
    .dropna(subset=['pos_in_range_100'])
    .query('pos_in_range_100 <= 0.25')
    .sort_values('pos_in_range_100', ascending=True)
    [[
        'exchange',
        'instrument',
        'range_perc_pred',
        'pos_in_range_100',
    ]]
    .head(20)
)

# Save CSV
bottom_lt_trend.to_csv(path_data_save / 'bottom_lt_trend.csv')

# Save Text File
txt_lt_down = ','.join([f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in bottom_lt_trend['instrument']])
with open(path_data_raw / 'Long-Term Underperformers.txt', 'w') as f:
    f.write(txt_lt_down)

## Combined Watchlist

In [55]:
sections = {
    'Long-Term Outperformers':[f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in top_lt_trend['instrument']],
    'Long-Term Underperformers':[f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in bottom_lt_trend['instrument']],
    'Short-Term Outperformers':[f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in top_st_trend['instrument']],
    'Short-Term Underperformers':[f'{dict_highest_volume.get(instrument).upper()}:{instrument}.P' for instrument in bottom_st_trend['instrument']],
}

filepath_combined_out_under = path_data_raw / 'Combined Out&Under-Performers.txt'
with open(filepath_combined_out_under, "w") as file:
    for section, symbols in sections.items():
        file.write(f"### {section}\n")  # Write section header
        file.write("\n".join(symbols) + "\n\n")  # Write symbols under 

# Push to s3

In [56]:
import boto3
import os
from botocore.exceptions import NoCredentialsError, ClientError
from dotenv import dotenv_values, find_dotenv

## Create Session

In [68]:
config = dotenv_values(find_dotenv())
aws_access_key_id = config.get("aws_access_key_id")
aws_secret_access_key = config.get("aws_secret_access_key")

session = boto3.Session(
    aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key
)
s3_client = session.client('s3')

## Save Data

In [69]:
filepath_recent = path_data_save / 'bnf_recent.csv'
df_recent.to_csv(filepath_recent, index=False)

## Upload Data

In [60]:
files_to_push = [
    'bnf_recent.csv',
    'big_boys.csv',
    'breakouts.csv',
    'breakdowns.csv',
    'dip_buys.csv',
    'rip_fades.csv',
    'top_st_trend.csv',
    'bottom_st_trend.csv',
    'top_lt_trend.csv',
    'bottom_lt_trend.csv',
]

for file in files_to_push:
    print(f'Pushing {file} to s3')
    file_path = path_data_save / file
    file_name = str(file_path.resolve())
    object_name = file
    # print(file_name)
    # print(object_name)
    bucket = "stellar-repos"
    upload_kwargs = {
        "ACL": "public-read",
        "ContentType": "text/html",
        "ContentDisposition": "inline",
    }
    
    
    try:
        response = s3_client.upload_file(
            file_name, bucket, object_name, ExtraArgs=upload_kwargs
        )
    except ClientError as e:
        print(e)

Pushing bnf_recent.csv to s3
Pushing big_boys.csv to s3
Pushing breakouts.csv to s3
Pushing breakdowns.csv to s3
Pushing dip_buys.csv to s3
Pushing rip_fades.csv to s3
Pushing top_st_trend.csv to s3
Pushing bottom_st_trend.csv to s3
Pushing top_lt_trend.csv to s3
Pushing bottom_lt_trend.csv to s3


In [76]:
response = s3_client.get_object(Bucket=bucket, Key='rip_fades.csv')
df_check_data = pd.read_csv(response.get("Body"), index_col=0)

In [77]:
df_check_data

,datetime,open,high,low,close,volume,volume_usd,range_perc,range_perc_ema_5,range_perc_ema_5_prev,pos_in_range_3,pos_in_range_7,pos_in_range_10,pos_in_range_30,pos_in_range_100,cmema_3_12,cmema_10_40,cmema_25_100,aroon_10,aroon_30,aroon_60,three_bar_triangle,high_3d,low_3d,range_perc_rel,range_perc_ema_5_log,range_perc_log_pred,range_perc_pred,low_distance
instrument,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CARVUSDT,2025-02-18,0.436900,0.470500,0.401600,0.410900,4962669.0,2.133824e+06,0.167681,0.088731,0.049256,0.134978,0.121094,0.121094,0.097492,0.037284,-0.690112,-2.551492,-3.412754,-66.666667,-48.275862,-67.796610,False,0.470500,0.401600,1.225029,-2.422147,-2.512917,0.081032,0.255077
GIGAUSDT,2025-02-18,0.029650,0.029780,0.020620,0.023220,453438940.0,1.170666e+07,0.394488,0.219587,0.132136,0.191599,0.149254,0.132721,0.035446,NaN,-0.884972,-1.554046,-1.306212,-77.777778,-100.000000,-77.966102,False,0.034190,0.020620,1.093752,-1.516008,-1.768859,0.170528,0.509923
AIOZUSDT,2025-02-18,0.425100,0.485200,0.381000,0.421000,19716871.0,8.440300e+06,0.247506,0.144655,0.093230,0.383877,0.383877,0.212993,0.066245,0.043056,-0.476758,-1.976384,-1.701760,-77.777778,-27.586207,-49.152542,False,0.485200,0.381000,0.976368,-1.933403,-2.111594,0.121045,0.656817
PRIMEUSDT,2025-02-18,6.382000,6.450000,5.277000,5.618000,507698.6,3.011541e+06,0.208793,0.123048,0.080176,0.239298,0.143943,0.125970,0.041525,0.025800,-0.926895,-2.115845,-1.760893,-77.777778,-100.000000,-72.881356,False,6.702000,5.277000,0.957121,-2.095178,-2.244433,0.105988,0.493284
IOTXUSDT,2025-02-18,0.020355,0.020367,0.017033,0.018308,555830309.0,1.056953e+07,0.182055,0.107640,0.070433,0.336967,0.235562,0.235562,0.064720,0.025409,-0.654272,-2.328347,-2.862875,-55.555556,-100.000000,-77.966102,False,0.020815,0.017033,0.949645,-2.228959,-2.354285,0.094961,0.646574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CRVUSDT,2025-02-18,0.516854,0.519088,0.467117,0.495205,302323020.8,1.510303e+08,0.104948,0.088056,0.079611,0.360891,0.278053,0.210927,0.172208,0.221816,-0.549353,-2.238341,-1.339919,-77.777778,-48.275862,-52.542373,False,0.544946,0.467117,0.276314,-2.429777,-2.519182,0.080525,0.644129
TOKENUSDT,2025-02-18,0.026537,0.026807,0.024284,0.025340,133724923.0,3.442364e+06,0.099561,0.083781,0.075891,0.319027,0.213398,0.194677,0.156831,0.058251,-0.609033,-2.871581,-3.779822,-77.777778,-48.275862,-42.372881,False,0.027594,0.024284,0.271465,-2.479546,-2.560049,0.077301,0.497285
AI16ZUSDT,2025-02-18,0.357427,0.363427,0.283164,0.316627,573377501.1,1.893072e+08,0.253496,0.214795,0.195444,0.218900,0.109920,0.096783,0.067224,NaN,-0.730674,-2.044476,-2.421499,-77.777778,-41.379310,0.000000,False,0.436036,0.283164,0.260072,-1.538072,-1.786977,0.167466,0.492041
